In [1]:
import numpy as np
import pandas as pd
import numpy as np
import os 
import time
from google.cloud import bigquery
import pandas_gbq
import glob

In [2]:
client = bigquery.Client()

In [10]:
query = """SELECT * FROM `repositoriodedadosgpsp.perfil_remuneracao.MILITARES_reserva_reforma_total`"""  

In [11]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df

Downloading: 100%|██████████|


,ano,mes,tipo_aposentadoria,org_lotacao,total
0,2024,4,Reforma,Comando da Marinha,1
1,2024,4,Reforma,Comando da Marinha,1
2,2024,4,Reforma,Comando da Marinha,1
3,2024,4,Reforma,Comando da Marinha,1
4,2024,4,Reforma,Comando da Marinha,1
...,...,...,...,...,...
996242,2020,4,Reforma Por Invalidez,Comando da Aeronáutica,1
996243,2020,4,Reforma Por Invalidez,Comando da Aeronáutica,1
996244,2020,4,Reforma Por Invalidez,Comando da Aeronáutica,1
996245,2020,4,Reforma Por Invalidez,Comando da Aeronáutica,1


In [12]:
df.columns

Index(['ano', 'mes', 'tipo_aposentadoria', 'org_lotacao', 'total'], dtype='object')

In [6]:
df['ano'].unique()

<IntegerArray>
[2024, 2021, 2022, 2023, 2020]
Length: 5, dtype: Int64

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 996247 entries, 0 to 996246
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   ano                 996247 non-null  Int64 
 1   mes                 996247 non-null  Int64 
 2   tipo_aposentadoria  996247 non-null  object
 3   org_lotacao         996247 non-null  object
 4   total               996247 non-null  Int64 
dtypes: Int64(3), object(2)
memory usage: 40.9+ MB


In [14]:
dir = 'G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS E CONHECIMENTO\\415 - Repositório de Dados\\Repositório Local\\Tabelas para o usuário baixar'

In [16]:
df

,ano,mes,tipo_aposentadoria,org_lotacao,total
0,2024,4,Reforma,Comando da Marinha,1
1,2024,4,Reforma,Comando da Marinha,1
2,2024,4,Reforma,Comando da Marinha,1
3,2024,4,Reforma,Comando da Marinha,1
4,2024,4,Reforma,Comando da Marinha,1
...,...,...,...,...,...
996242,2020,4,Reforma Por Invalidez,Comando da Aeronáutica,1
996243,2020,4,Reforma Por Invalidez,Comando da Aeronáutica,1
996244,2020,4,Reforma Por Invalidez,Comando da Aeronáutica,1
996245,2020,4,Reforma Por Invalidez,Comando da Aeronáutica,1


In [15]:
os.chdir(dir) 

In [17]:
df.to_csv('MILITARES_reserva_reforma_total.csv', sep=';', encoding='latin-1', index=False)

-----------------------------------------------------------

Testes

In [14]:
df = pd.read_csv("SIAPE_ingresso_aposentadoria_media.csv", encoding="latin-1", sep=';', decimal=',')
df

,tipo_aposentadoria,decada_aposentadoria,categoria_ministerios,media_contribuicao,quantidade_vinculos
0,Outros,1980,"Economia, Finanças, Planejamento, Previdência ...",26.0,3
1,Outros,1980,Educação e Ciência e Tecnologia,19.7,6
2,Outros,1980,Saúde,19.2,4
3,Outros,1990,"Cultura, Turismo e Esportes",12.0,1
4,Outros,1990,"Economia, Finanças, Planejamento, Previdência ...",26.6,5
...,...,...,...,...,...
200,Aposentadoria por incapacidade,2020,"Justiça, Segurança e Órgãos de Controle e Defesa",23.0,152
201,Aposentadoria por incapacidade,2020,"Meio ambiente, Agricultura e Agropecuária",27.2,37
202,Aposentadoria por incapacidade,2020,Presidência da República,20.1,51
203,Aposentadoria por incapacidade,2020,Relações Exteriores,29.9,13


Automatizando troca de csv

In [45]:
##os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\\Repositório Local\\Tabelas para o usuário baixar")

In [46]:
## baixando os atuais com , e exportando com ";"

In [47]:
## listando todos os itens do diretório
##len(glob.glob('*.csv'))

In [48]:
## Serve apenas na primeira vez . 

##for i in list(range(0,len(glob.glob('*.csv')))):
    
   # df= pd.read_csv(glob.glob('*.csv')[i])
    #df.to_csv(glob.glob('*.csv')[i],index=False, encoding="latin-1", sep=';', decimal=',')